In [6]:
import duckdb
import torch
import torch.nn.functional as F
import torch_frame as tf
from torch_frame.data import Dataset
from torch_frame.utils import infer_df_stype
from torch_frame.data.loader import DataLoader
from torch_frame.nn.encoder import EmbeddingEncoder, LinearEncoder
from torch_frame.nn.models import FTTransformer   # any backbone works

# -----------------------------------------------------------
# 1.  Read & sample exactly the same way you did
# -----------------------------------------------------------
file_path = "../data/parcel_tracking_output_data.parquet"    # (or .xlsx → convert once)
df = duckdb.sql(f"""
    SELECT * FROM '{file_path}'
    USING SAMPLE reservoir(100 ROWS)
    REPEATABLE (100)
""").df()

# Targets ----------------------------------------------------
ETA_COL   = "total_hours_from_receiving_to_last_success_delivery"
POD_COL   = "is_successful_delivery"

# Cast targets to numeric
df[ETA_COL] = df[ETA_COL].astype(float)
df[POD_COL] = df[POD_COL].astype(int)

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [13]:
# -----------------------------------------------------------
# 2.  Build two Dataset objects (one per task)
# -----------------------------------------------------------
col_to_stype = infer_df_stype(df)                           # auto‑detect stypes
col_to_stype[ETA_COL] = tf.numerical                       # overwrite to be safe
col_to_stype[POD_COL] = tf.categorical                     # binary → categorical

eta_ds = Dataset(df, col_to_stype=col_to_stype,
                 target_col=ETA_COL)
pod_ds = Dataset(df, col_to_stype=col_to_stype,
                 target_col=POD_COL)

# Materialise once so we can reuse stats & mappings
eta_ds.materialize()              # gives .tensor_frame, .col_stats
pod_ds.materialize()

# Split ------------------------------------------------------
train_eta = eta_ds[:0.8]    # same slice trick as in quick‑tour :contentReference[oaicite:3]{index=3}
val_eta   = eta_ds[0.8:]
train_pod = pod_ds[:0.8]
val_pod   = pod_ds[0.8:]

# -----------------------------------------------------------
# 3.  Mini‑batch loaders
# -----------------------------------------------------------
BATCH = 256
train_eta_loader = DataLoader(train_eta, batch_size=BATCH, shuffle=True)
val_eta_loader   = DataLoader(val_eta,   batch_size=BATCH)

train_pod_loader = DataLoader(train_pod, batch_size=BATCH, shuffle=True)
val_pod_loader   = DataLoader(val_pod,   batch_size=BATCH)

# -----------------------------------------------------------
# 4.  Common stype‑wise encoders
# -----------------------------------------------------------
stype_enc = {
    tf.stype.categorical: EmbeddingEncoder(),
    tf.stype.numerical:   LinearEncoder(),
}

# -----------------------------------------------------------
# 5‑A.  ETA regressor  (FT‑Transformer, 1 output)
# -----------------------------------------------------------
eta_model = FTTransformer(
    channels=32,
    out_channels=1,
    num_layers=3,
    col_stats=train_eta.col_stats,
    col_names_dict=train_eta.tensor_frame.col_names_dict,
    stype_encoder_dict=stype_enc,
)
opt_eta = torch.optim.AdamW(eta_model.parameters(), lr=1e-3)

# -----------------------------------------------------------
# 5‑B.  POD classifier (same backbone, sigmoid output)
# -----------------------------------------------------------
pod_model = FTTransformer(
    channels=32,
    out_channels=1,
    num_layers=3,
    col_stats=train_pod.col_stats,
    col_names_dict=train_pod.tensor_frame.col_names_dict,
    stype_encoder_dict=stype_enc,
)
opt_pod = torch.optim.AdamW(pod_model.parameters(), lr=1e-3)

In [18]:
for tf_batch in train_eta_loader:
    print(tf_batch)
    break

TensorFrame(
  num_cols=26,
  num_rows=80,
  categorical (6): ['Rush_hour', 'is_successful_delivery', 'is_successful_delivery_at_first_time', 'last_delivery_datetime_is_non_working_day', 'parcel_category_id', 'received_datetime_is_non_working_day'],
  numerical (20): ['customer_id', 'delivery_post_office_address_latitude', 'delivery_post_office_address_longtitude', 'delivery_post_office_id', 'distance_delivery_post_office_recipient_address', 'distance_received_post_office_delivery_post_office', 'distance_received_post_office_recipient_address', 'last_delivery_datetime_day', 'last_delivery_datetime_hour', 'last_delivery_datetime_month', 'parcel_id', 'received_datetime_day', 'received_datetime_hour', 'received_datetime_month', 'received_post_office_address_latitude', 'received_post_office_address_longtitude', 'received_post_office_id', 'recipient_address_latitude', 'recipient_address_longtitude', 'total_hours_from_receiving_to_last_failed_delivery'],
  has_target=True,
  device='cpu',
)


In [17]:
# -----------------------------------------------------------
# 6.  Plain PyTorch training loops (no Lightning/Rich headaches)
# -----------------------------------------------------------
def train_epoch(model, loader, optimizer, is_regression: bool):
    model.train()
    total = 0; loss_sum = 0
    for tf_batch in loader:
        print(tf_batch.feat_dict)
        pred = model(tf_batch)
        target = tf_batch.y.float()
        loss = F.smooth_l1_loss(pred.squeeze(), target) if is_regression \
               else F.binary_cross_entropy_with_logits(pred.squeeze(), target)
        optimizer.zero_grad(); loss.backward(); optimizer.step()
        loss_sum += loss.item() * len(target); total += len(target)
    return loss_sum / total

for epoch in range(1, 31):
    eta_loss = train_epoch(eta_model, train_eta_loader, opt_eta, True)
    pod_loss = train_epoch(pod_model, train_pod_loader, opt_pod, False)
    print(f"Epoch {epoch:02d} | ETA MAE-ish {eta_loss:.3f} | POD BCE {pod_loss:.3f}")

{<stype.categorical: 'categorical'>: tensor([[0, 1, 1, 0, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 1, 1, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [0, 1, 1, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 0],
        [1, 0, 0, 0, 1, 0],
        [1, 1, 1, 1, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [1, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 1],
        [1, 0, 0, 0, 2, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0],
        [0, 0, 1, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 1, 1, 0, 0, 1],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0, 0],
        [0, 0, 0, 1, 0, 1],
        [0, 0, 1, 0, 0, 0],
        [1, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 1, 0],
        [1,

RuntimeError: The size of tensor a (21) must match the size of tensor b (20) at non-singleton dimension 1

In [ ]:
# -----------------------------------------------------------
# 7.  Validation & inference
# -----------------------------------------------------------
@torch.no_grad()
def evaluate(model, loader, is_regression):
    model.eval()
    outs, ys = [], []
    for tf_batch in loader:
        outs.append(model(tf_batch).cpu())
        ys.append(tf_batch.y.float().cpu())
    pred = torch.cat(outs).squeeze()
    y    = torch.cat(ys).squeeze()
    if is_regression:
        return torch.mean(torch.abs(pred - y)).item()      # MAE
    else:
        prob = torch.sigmoid(pred)
        acc  = ((prob > 0.5) == y.bool()).float().mean()
        return acc.item()

print("ETA MAE (val):", evaluate(eta_model, val_eta_loader, True))
print("POD ACC (val):", evaluate(pod_model, val_pod_loader, False))

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

RuntimeError: The size of tensor a (21) must match the size of tensor b (20) at non-singleton dimension 1

---

In [1]:
import duckdb
import pandas as pd

file_path = "../data/parcel_tracking_output_data.parquet"         # or .xlsx after conversion
df = duckdb.sql(f"""
    SELECT * FROM '{file_path}'
    USING SAMPLE reservoir(100 ROWS)
    REPEATABLE (100)
""").df()

df = df.dropna(subset=[
    "total_hours_from_receiving_to_last_success_delivery",
    "is_successful_delivery",
])

# Cast targets
df["is_successful_delivery"] = df["is_successful_delivery"].astype(int)
df["total_hours_from_receiving_to_last_success_delivery"] = (
    df["total_hours_from_receiving_to_last_success_delivery"].astype(float)
)

# Identify feature types
target_reg = ["total_hours_from_receiving_to_last_success_delivery"]
target_cls = ["is_successful_delivery"]
categorical_cols = [c for c in df.select_dtypes("object").columns]
continuous_cols  = [c for c in df.columns if c not in categorical_cols + target_reg + target_cls]

FloatProgress(value=0.0, layout=Layout(width='auto'), style=ProgressStyle(bar_color='black'))

In [2]:
from pytorch_tabular import TabularModel
from pytorch_tabular.models import NodeConfig
from pytorch_tabular.config import DataConfig, TrainerConfig, OptimizerConfig

data_reg = DataConfig(
    target           = target_reg,
    continuous_cols  = continuous_cols,
    categorical_cols = categorical_cols,
)

model_reg = NodeConfig(
    num_layers   = 4,
    num_trees    = 8,
    depth        = 3,
    task         = "regression",
    learning_rate= 1e-3,
    metrics      = ["mean_absolute_error"],
)

trainer_reg = TrainerConfig(
    max_epochs        = 50,
    batch_size        = 10,
    auto_lr_find      = False,          # keep LR‑Finder off (previous fix)
    accelerator       = "cpu",
    trainer_kwargs={"enable_progress_bar": False}
)

optim_reg = OptimizerConfig(
    optimizer        = "AdamW",
    optimizer_params = {"weight_decay": 1e-5},
)

eta_model = TabularModel(
    data_config      = data_reg,
    model_config     = model_reg,
    trainer_config   = trainer_reg,
    optimizer_config = optim_reg,
)

eta_model.fit(train=df, validation=df.sample(frac=0.15, random_state=42))

2025-07-26 18:17:03,705 - {pytorch_tabular.tabular_model:140} - INFO - Experiment Tracking is turned off

Seed set to 42


2025-07-26 18:17:03,754 - {pytorch_tabular.tabular_model:524} - INFO - Preparing the DataLoaders

2025-07-26 18:17:03,758 - {pytorch_tabular.tabular_datamodule:499} - INFO - Setting up the datamodule for          
regression task

/home/nguynking/eta/.venv/lib/python3.12/site-packages/pytorch_tabular/tabular_datamodule.py:368: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '[ 0.56384729 -1.44989302  0.56384729  0.56384729  0.56384729  0.56384729
 -1.44989302 -2.45676318 -1.44989302  0.56384729 -1.44989302  0.56384729
 -1.44989302  0.56384729  0.56384729 -2.45676318 -2.45676318  0.56384729
 -1.44989302  0.56384729  0.56384729  0.56384729 -1.44989302  0.56384729
 -2.45676318  0.56384729  0.56384729 -1.44989302 -1.44989302  0.56384729
  0.56384729  0.56384729  0.56384729 -1.44989302  0.56384729  0.56384729
  0.56384729  0.56384729  0.56384729  0.56384729 -1.44989302 -1.44989302
  0.56384729  0.56384729  0.56384729  0.56384729  0.56384729  0.56384729
  0.56384729  0.56384729  0.56384729 -2.45676318  0.56384729  0.56384729
  0.56384729  0.56384729  0.56384729  0.56384729  0.56384729  0.56384729
  0.56384729  0.56384729  0.56384729  0.56384729  0.56

...

2025-07-26 18:17:04,277 - {pytorch_tabular.tabular_model:652} - INFO - Training Started

/home/nguynking/eta/.venv/lib/python3.12/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:639: Checkpoint directory /home/nguynking/eta/notebook/saved_models exists and is not empty.

  | Name             | Type             | Params
------------------------------------------------------
0 | _backbone        | NODEBackbone     | 8.4 K 
1 | _embedding_layer | Embedding1dLayer | 50    
2 | _head            | Lambda           | 0     
3 | loss             | MSELoss          | 0     
------------------------------------------------------
8.3 K     Trainable params
196       Non-trainable params
8.5 K     Total params
0.034     Total estimated model params size (MB)
/home/nguynking/eta/.venv/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=7` in the `DataLoader` to improve performance.
/hom

2025-07-26 18:17:18,952 - {pytorch_tabular.tabular_model:663} - INFO - Training the model completed

2025-07-26 18:17:18,954 - {pytorch_tabular.tabular_model:1489} - INFO - Loading the best model

UnpicklingError: Weights only load failed. This file can still be loaded, to do so you have two options, [1mdo those steps only if you trust the source of the checkpoint[0m. 
	(1) In PyTorch 2.6, we changed the default value of the `weights_only` argument in `torch.load` from `False` to `True`. Re-running `torch.load` with `weights_only` set to `False` will likely succeed, but it can result in arbitrary code execution. Do it only if you got the file from a trusted source.
	(2) Alternatively, to load with `weights_only=True` please check the recommended steps in the following error message.
	WeightsUnpickler error: Unsupported global: GLOBAL omegaconf.dictconfig.DictConfig was not an allowed global by default. Please use `torch.serialization.add_safe_globals([omegaconf.dictconfig.DictConfig])` or the `torch.serialization.safe_globals([omegaconf.dictconfig.DictConfig])` context manager to allowlist this global if you trust this class/function.

Check the documentation of torch.load to 

In [ ]:
import torch
torch.load("saved_models/regression-5_epoch=49-valid_loss=678.95.ckpt"  )

In [ ]:
# from pytorch_tabular import TabularModel
from pytorch_tabular.config import DataConfig, TrainerConfig, OptimizerConfig
from pytorch_tabular.models import NodeConfig                # fast MLP trunk
from pytorch_tabular.models.common.heads import (
    RegressionHeadConfig,
    ClassificationHeadConfig,
    MultiTaskHeadConfig,
)

# 1) DataConfig  (tells PT‑Tabular what columns mean what)
data_config = DataConfig(
    target=target_cols,                    # two targets
    continuous_cols=continuous_cols,
    categorical_cols=categorical_cols,
    continuous_feature_transform="standardize",
    categorical_encoder="embedding",       # learn embeddings for high‑card cat
)

# 2) Shared trunk  (Node = residual MLP, similar to FT‑Transformer without attention)
model_config = NodeConfig(
    task="multitask",                      # important!
    num_layers=4,
    num_trees=8,                           # Node’s internal structure
    depth=2,
    learning_rate=1e-3,
)

# 3) Two heads
reg_head = RegressionHeadConfig(
    head="eta",
    output_dim=1,
    loss="SmoothL1Loss",
    metric_list=["mae", "rmse"],
)

cls_head = ClassificationHeadConfig(
    head="pod",
    output_dim=1,
    loss="BCEWithLogitsLoss",
    metric_list=["accuracy", "auc"],
)

multi_head = MultiTaskHeadConfig(
    heads=[reg_head, cls_head],
    head_weights=[1.0, 1.0],               # α, β  ← tune to business cost
)

# 4) Trainer & optimiser
trainer_config = TrainerConfig(
    auto_lr_find=True,
    max_epochs=50,
    batch_size=2048,
    early_stopping="val_loss",
    gpus=1,                                # 0 for CPU
    precision=16,                          # AMP
)

optimizer_config = OptimizerConfig(
    optimizer="AdamW",
    weight_decay=1e-5,
)


---

In [ ]:
# dataloader.py
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

# ──────────────────────────────────────────────────────────────
# 1.  Low‑level Dataset
# ──────────────────────────────────────────────────────────────
class ParcelDataset(Dataset):
    def __init__(self, df, num_cols, scaler):
        self.y_eta  = df["total_hours_from_receiving_to_last_success_delivery"].values.astype("float32")
        self.y_pod  = df["is_successful_delivery"].values.astype("float32")

        # numeric → z‑score
        self.X = scaler.transform(df[num_cols]).astype("float32")

    def __len__(self):
        return len(self.y_eta)

    def __getitem__(self, idx):
        return (
            torch.from_numpy(self.X[idx]),          # features
            torch.tensor(self.y_eta[idx]),          # ETA (regression)
            torch.tensor(self.y_pod[idx])           # POD success flag
        )


# ──────────────────────────────────────────────────────────────
# 2.  Convenience factory – returns train/val DataLoaders
# ──────────────────────────────────────────────────────────────
def make_dataloaders(
        file_path: str | Path,
        num_cols: list[str],
        batch_size: int = 10,
        val_split: float = 0.15,
        seed: int = 42,
        num_workers: int = 0            # >0 if running locally with CPU cores
):
    df = duckdb.sql(f"""
        SELECT * FROM '{file_path}'
        USING SAMPLE reservoir(100 ROWS)
        REPEATABLE (100)
    """).df().dropna(subset=["total_hours_from_receiving_to_last_success_delivery", "is_successful_delivery"])
                                            
    # train/val split (stratify by class so POD imbalance is preserved)
    train_df, val_df = train_test_split(
        df,
        test_size=val_split,
        stratify=df["is_successful_delivery"],
        random_state=seed
    )

    # fit scalers *only on train*  → avoids target leakage
    scaler  = StandardScaler().fit(train_df[num_cols])

    train_ds = ParcelDataset(train_df, num_cols, scaler)
    val_ds   = ParcelDataset(val_df,   num_cols, scaler)

    train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True,  num_workers=num_workers, pin_memory=True)
    val_dl   = DataLoader(val_ds,   batch_size=batch_size, shuffle=False, num_workers=num_workers, pin_memory=True)

    input_dim = train_ds.X.shape[1]    # feed into MultiTaskNet(input_dim)

    return train_dl, val_dl, input_dim, scaler

In [ ]:
train_loader, val_loader, input_dim, scaler = make_dataloaders(
    file_path=Path("../data/parcel_tracking_output_data.parquet"),
    num_cols=parcel_df.select_dtypes(include='number').drop(columns=["total_hours_from_receiving_to_last_success_delivery", "is_successful_delivery", "parcel_id"]).columns.tolist()
)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

class MultiTaskNet(nn.Module):
    def __init__(self, d_in, h=128):
        super().__init__()
        self.shared = nn.Sequential(
            nn.Linear(d_in, h), nn.BatchNorm1d(h), nn.ReLU(),
            nn.Linear(h, h), nn.ReLU()
        )
        self.reg  = nn.Sequential(nn.Linear(h, h//2), nn.ReLU(), nn.Linear(h//2, 1))
        self.cls  = nn.Sequential(nn.Linear(h, h//2), nn.ReLU(), nn.Linear(h//2, 1), nn.Sigmoid())

    def forward(self, x):
        z   = self.shared(x)
        eta = self.reg(z)        # hours
        pod = self.cls(z)        # probability
        return eta, pod


In [ ]:
model  = MultiTaskNet(input_dim)
opt    = optim.AdamW(model.parameters(), lr=3e-4)
for X_batch, y_eta, y_pod in train_loader:
    eta_hat, pod_hat = model(X_batch)
    loss = alpha * nn.functional.smooth_l1_loss(eta_hat.squeeze(), y_eta) + beta * nn.functional.binary_cross_entropy(pod_hat.squeeze(), y_pod)
    opt.zero_grad()
    loss.backward()
    pt.step()